# Task 2: Exploratory Data Analysis — Financial Inclusion in Ethiopia

**Objective:** Analyze patterns and factors influencing financial inclusion.

1. Dataset overview & temporal coverage
2. Access analysis (account ownership trajectory, growth rates, gender)
3. Usage analysis (digital payments, mobile money)
4. Infrastructure and enablers
5. Event timeline and overlay on trends
6. Correlation analysis
7. Key insights & data quality assessment

In [ ]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent if "notebooks" in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(ROOT))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data import load_unified_data, load_reference_codes, load_guide, enrich_unified_data
from src.analysis import get_access_series, get_usage_series, get_events_timeline

pd.set_option("display.max_columns", 20)
sns.set_theme(style="whitegrid")

In [ ]:
df = load_unified_data()
df = enrich_unified_data(df)
obs = df[df["record_type"] == "observation"].copy()
events = df[df["record_type"] == "event"].copy()
impact = df[df["record_type"] == "impact_link"].copy()
obs["observation_date"] = pd.to_datetime(obs["observation_date"])
obs["year"] = obs["observation_date"].dt.year
events["observation_date"] = pd.to_datetime(events["observation_date"])

---
## 1. Dataset Overview

### 1.1 Summarize by record_type, pillar, source_type

In [ ]:
print("Record types:")
print(df["record_type"].value_counts().to_string())
print()
print("Observations by pillar:")
print(obs["pillar"].value_counts().to_string())
print()
print("Observations by source_type:")
print(obs["source_type"].value_counts().to_string())

### 1.2 Temporal coverage: which years have data for which indicators?

In [ ]:
ind_year = obs.pivot_table(
    index="indicator_code", columns="year", values="value_numeric", aggfunc="mean"
)
ind_year_bin = (ind_year.notna()).astype(int)
plt.figure(figsize=(12, 10))
sns.heatmap(ind_year_bin, cmap="Blues", cbar_kws={"label": "Has data (1)"})
plt.title("Temporal coverage: indicators x year")
plt.tight_layout()
plt.show()

### 1.3 Data quality: distribution of confidence levels

In [ ]:
conf = obs["confidence"].value_counts()
plt.figure(figsize=(6, 4))
conf.plot(kind="bar", color=["#2ecc71", "#f39c12", "#e74c3c", "#95a5a6"])
plt.title("Observation confidence distribution")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
print(conf.to_string())

### 1.4 Gaps: indicators with sparse coverage

In [ ]:
n_obs_per_ind = obs.groupby("indicator_code").agg(
    n_obs=("record_id", "count"), years=("year", "nunique")
).sort_values("n_obs")
print("Indicators by number of observations (sparse at top):")
print(n_obs_per_ind.to_string())

---
## 2. Access Analysis

### 2.1 Account ownership trajectory (2011–2024)

In [ ]:
acc_nat = obs[(obs["indicator_code"] == "ACC_OWNERSHIP") & (obs["gender"] == "all")].sort_values("observation_date")
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(acc_nat["observation_date"], acc_nat["value_numeric"], "o-", color="#2980b9", linewidth=2, markersize=10)
ax.set_xlabel("Year")
ax.set_ylabel("Account ownership (%)")
ax.set_title("Ethiopia: Account ownership trajectory (2011–2024)")
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2.2 Growth rates between survey years

In [ ]:
acc_ts = acc_nat.sort_values("observation_date")
acc_ts = acc_ts.drop_duplicates(subset=["year"], keep="first")
acc_ts["prev"] = acc_ts["value_numeric"].shift(1)
acc_ts["growth_pp"] = acc_ts["value_numeric"] - acc_ts["prev"]
acc_ts["growth_pct"] = (acc_ts["value_numeric"] - acc_ts["prev"]) / acc_ts["prev"] * 100
print("Account ownership: level and growth between survey years")
print(acc_ts[["year", "value_numeric", "growth_pp", "growth_pct"]].to_string(index=False))

fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(acc_ts["year"].astype(str), acc_ts["growth_pp"], color="#27ae60", alpha=0.8)
ax.axhline(0, color="black", linewidth=0.5)
ax.set_xlabel("Year (period end)")
ax.set_ylabel("Growth (percentage points)")
ax.set_title("Account ownership: change vs previous survey")
plt.tight_layout()
plt.show()

### 2.3 Gender gap (male vs female ownership)

In [ ]:
acc_gender = obs[(obs["indicator_code"] == "ACC_OWNERSHIP") & (obs["gender"].isin(["male", "female"]))]
if len(acc_gender) >= 2:
    pivot = acc_gender.pivot_table(index="year", columns="gender", values="value_numeric")
    pivot["gap_pp"] = pivot["male"] - pivot["female"]
    print("Account ownership by gender and gap (pp):")
    print(pivot.to_string())
    fig, ax = plt.subplots(figsize=(8, 4))
    x = pivot.index.astype(str)
    ax.bar(np.arange(len(x)) - 0.2, pivot["male"], width=0.4, label="Male", color="#3498db")
    ax.bar(np.arange(len(x)) + 0.2, pivot["female"], width=0.4, label="Female", color="#e74c3c")
    ax.set_xticks(np.arange(len(x)))
    ax.set_xticklabels(x)
    ax.set_ylabel("%")
    ax.set_title("Account ownership by gender")
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Insufficient gender-disaggregated data for ACC_OWNERSHIP.")

gen_gap = obs[obs["indicator_code"] == "GEN_GAP_ACC"]
if len(gen_gap) > 0:
    print("Gender gap (GEN_GAP_ACC) over time:")
    print(gen_gap[["observation_date", "value_numeric"]].to_string(index=False))

### 2.4 Urban vs rural
*(Dataset has location="national" only; no urban/rural disaggregation in observations.)*

In [ ]:
print("Location values in observations:", obs["location"].dropna().unique().tolist())

### 2.5 Slowdown analysis (annualized growth and interpolated YoY)

This section quantifies whether account ownership growth has slowed by:
- Annualizing the change between survey years (percentage points per year)
- Interpolating annual values (linear) to visualize year-on-year momentum

In [ ]:
acc_nat_s = obs[(obs["indicator_code"] == "ACC_OWNERSHIP") & (obs["gender"] == "all")].sort_values("observation_date")
acc_survey = acc_nat_s.drop_duplicates(subset=["year"], keep="first")[["year", "value_numeric"]].sort_values("year").copy()
acc_survey["prev_year"] = acc_survey["year"].shift(1)
acc_survey["prev_value"] = acc_survey["value_numeric"].shift(1)
acc_survey["delta_pp"] = acc_survey["value_numeric"] - acc_survey["prev_value"]
acc_survey["years_gap"] = acc_survey["year"] - acc_survey["prev_year"]
acc_survey["annualized_pp_per_year"] = acc_survey["delta_pp"] / acc_survey["years_gap"]
print("Account ownership annualized change (pp/year) between observation years")
display(acc_survey[["prev_year", "year", "prev_value", "value_numeric", "delta_pp", "years_gap", "annualized_pp_per_year"]])

years_full = np.arange(int(acc_survey["year"].min()), int(acc_survey["year"].max()) + 1)
vals_interp = np.interp(years_full, acc_survey["year"].to_numpy(), acc_survey["value_numeric"].to_numpy())
yoy_interp = pd.Series(vals_interp, index=years_full).diff(1)

fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=False)
ax = axes[0]
bar = acc_survey.dropna(subset=["annualized_pp_per_year"]).copy()
ax.bar(bar["year"].astype(int).astype(str), bar["annualized_pp_per_year"], color="#f39c12", alpha=0.85)
ax.axhline(0, color="black", linewidth=0.6)
ax.set_ylabel("pp per year")
ax.set_title("Account ownership slowdown: annualized growth between observation years")
ax.grid(True, axis="y", alpha=0.25)

ax = axes[1]
ax.plot(years_full, vals_interp, "o-", color="#2980b9", linewidth=2, label="Interpolated level")
ax2 = ax.twinx()
ax2.plot(years_full, yoy_interp, "s--", color="#c0392b", linewidth=1.8, label="Interpolated YoY change (pp)")
ax.set_xlabel("Year")
ax.set_ylabel("Account ownership (%)")
ax2.set_ylabel("YoY change (pp)")
ax.set_title("Account ownership slowdown: interpolated annual momentum")
ax.grid(True, alpha=0.25)

lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc="upper left")
plt.tight_layout()
plt.show()

---
## 3. Usage (Digital Payments) Analysis

### 3.1 Mobile money account penetration (2014–2024)

In [ ]:
mm_acc = obs[obs["indicator_code"] == "ACC_MM_ACCOUNT"].sort_values("observation_date")
if len(mm_acc) > 0:
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(mm_acc["observation_date"], mm_acc["value_numeric"], "s-", color="#8e44ad", linewidth=2)
    ax.set_xlabel("Year")
    ax.set_ylabel("Mobile money account (%)")
    ax.set_title("Mobile money account penetration")
    ax.set_ylim(0, 15)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    print(mm_acc[["observation_date", "value_numeric", "unit"]].to_string(index=False))
else:
    print("No ACC_MM_ACCOUNT data.")

### 3.2 Digital payment adoption

In [ ]:
dig = obs[obs["indicator_code"] == "USG_DIGITAL_PAY"].sort_values("observation_date")
if len(dig) > 0:
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(dig["observation_date"], dig["value_numeric"], "o-", color="#16a085", linewidth=2)
    ax.set_xlabel("Year")
    ax.set_ylabel("%")
    ax.set_title("Digital payment adoption (made or received digital payment, past 12 months)")
    ax.set_ylim(0, 50)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
print("Registered vs active: Telebirr/M-Pesa user counts vs Findex MM account % — different denominators; operator data = registered, Findex = usage in past 12 months.")

In [ ]:
# Mobile money: penetration + registered vs active comparisons (when available)

mm_related = obs[
    obs["indicator_code"].astype(str).str.contains("MM|MOBILE|TELEBIRR|MPESA|ACTIVE|REGISTER", case=False, na=False)
    | obs["indicator"].astype(str).str.contains("mobile money|telebirr|m-pesa|mpesa|registered|active", case=False, na=False)
].copy()

print("Mobile-money-related indicators available in the dataset:")
display(
    mm_related[["indicator_code", "indicator", "unit", "source_type"]]
    .drop_duplicates()
    .sort_values(["indicator_code", "indicator"])
)

# Prefer explicit codes if present; otherwise fall back to keyword-matched candidates.
registered_codes = [
    c
    for c in obs["indicator_code"].dropna().unique().tolist()
    if any(k in str(c).upper() for k in ["REG", "REGISTER", "REGISTERED"])
]
active_codes = [
    c
    for c in obs["indicator_code"].dropna().unique().tolist()
    if any(k in str(c).upper() for k in ["ACTIVE"]) and c != "USG_DIGITAL_PAY"
]

def _series(code: str) -> pd.DataFrame:
    s = obs[obs["indicator_code"] == code].dropna(subset=["observation_date", "value_numeric"]).sort_values("observation_date")
    return s[["observation_date", "year", "value_numeric", "unit", "indicator", "source_type"]].copy()

# Build a comparison set.
series_to_plot = []
labels = []

if "ACC_MM_ACCOUNT" in obs["indicator_code"].unique():
    series_to_plot.append(_series("ACC_MM_ACCOUNT"))
    labels.append("Mobile money account (% adults)")

if "USG_ACTIVE_RATE" in obs["indicator_code"].unique():
    series_to_plot.append(_series("USG_ACTIVE_RATE"))
    labels.append("Active use rate (proxy; %)")

# Registered codes: only add those that have numeric values.
for c in registered_codes:
    s = _series(c)
    if len(s) > 0:
        series_to_plot.append(s)
        labels.append(f"Registered ({c})")

if len(series_to_plot) == 0:
    print("No mobile-money registered/active indicators found beyond ACC_MM_ACCOUNT/USG_ACTIVE_RATE.")
else:
    # If units mix counts and percentages, use a secondary axis for non-% units.
    units = [str(s["unit"].iloc[0]) if len(s) else "" for s in series_to_plot]
    is_percent = [u.strip() == "%" or "percent" in u.lower() for u in units]

    fig, ax = plt.subplots(figsize=(10, 5))
    ax2 = None

    for s, lab, pct in zip(series_to_plot, labels, is_percent):
        if len(s) == 0:
            continue
        target_ax = ax if pct else (ax2 if ax2 is not None else ax.twinx())
        if (not pct) and ax2 is None:
            ax2 = target_ax

        target_ax.plot(s["observation_date"], s["value_numeric"], marker="o", linewidth=2, label=lab)

    ax.set_title("Mobile money: penetration and registered vs active (where available)")
    ax.set_xlabel("Year")
    ax.set_ylabel("% (survey-style)")
    ax.grid(True, alpha=0.25)

    if ax2 is not None:
        ax2.set_ylabel("Non-% unit (operator/regulator counts)")

    lines1, labels1 = ax.get_legend_handles_labels()
    if ax2 is not None:
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax.legend(lines1 + lines2, labels1 + labels2, loc="upper left")
    else:
        ax.legend(loc="upper left")

    plt.tight_layout()
    plt.show()

print(
    "Interpretation note: operator 'registered' counts and survey '% adults' are not directly comparable; "
    "registered != active, and denominators differ (subscribers vs adults)."
)

---
## 4. Infrastructure and Enablers

In [ ]:
infra_codes = ["ACC_4G_COV", "ACC_MOBILE_PEN", "ACC_ATM_DENSITY", "ACC_BRANCH_DENSITY", "ACC_FAYDA"]
infra = obs[obs["indicator_code"].isin(infra_codes)].copy()
infra = infra.dropna(subset=["value_numeric"])
if len(infra) > 0:
    fig, ax = plt.subplots(figsize=(10, 5))
    for code in infra["indicator_code"].unique():
        sub = infra[infra["indicator_code"] == code].sort_values("observation_date")
        ax.plot(sub["observation_date"], sub["value_numeric"], "o-", label=code, alpha=0.8)
    ax.set_xlabel("Date")
    ax.set_ylabel("Value")
    ax.set_title("Infrastructure & enablers (4G, mobile pen, Fayda; ATM/branch if present)")
    ax.legend(bbox_to_anchor=(1.02, 1))
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
print("Leading indicators: 4G coverage and mobile penetration enable account opening and usage; Fayda supports KYC and inclusion.")

---
## 5. Event Timeline and Overlay

In [ ]:
ev = get_events_timeline(df)
fig, ax = plt.subplots(figsize=(12, max(6, len(ev) * 0.4)))
y_pos = np.arange(len(ev))
dates = pd.to_datetime(ev["observation_date"])
ax.scatter(dates, y_pos, s=80, color="steelblue", zorder=2)
ax.set_yticks(y_pos)
ax.set_yticklabels(ev["indicator"].str[:55] + " (" + ev["category"].astype(str) + ")", fontsize=9)
ax.set_xlabel("Date")
ax.set_title("Cataloged events timeline")
ax.set_xlim(pd.Timestamp("2020-01-01"), pd.Timestamp("2026-01-01"))
ax.set_ylim(-0.5, len(ev) - 0.5)
ax.grid(True, axis="x", alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
acc_nat = obs[(obs["indicator_code"] == "ACC_OWNERSHIP") & (obs["gender"] == "all")].sort_values("observation_date")
ax.plot(acc_nat["observation_date"], acc_nat["value_numeric"], "o-", label="Account ownership", color="#2980b9", linewidth=2)
ax.set_ylim(0, 100)
for _, r in ev.iterrows():
    ax.axvline(r["observation_date"], color="red", alpha=0.4, linestyle="--")
    ax.text(r["observation_date"], 92, r["indicator"][:25] + "…", rotation=90, fontsize=8)
ax.set_xlabel("Year")
ax.set_ylabel("Account ownership (%)")
ax.set_title("Account ownership with events overlay")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.3 Indicator trends with major events overlaid (Access, Usage, Enablers)

To avoid clutter, the overlay focuses on a small subset of major events (filtered by category/keywords) rather than every cataloged event.

---
## 6. Correlation Analysis

In [ ]:
# 5.3 Major-events overlay on key indicator trends

# Choose a small set of indicators to plot (only those that exist in the dataset)
preferred_codes = [
    "ACC_OWNERSHIP",
    "USG_DIGITAL_PAY",
    "ACC_MM_ACCOUNT",
    "ACC_MOBILE_PEN",
    "ACC_4G_COV",
    "ACC_FAYDA",
]

available_codes = set(obs["indicator_code"].dropna().unique().tolist())
codes_to_plot = [c for c in preferred_codes if c in available_codes]

if len(codes_to_plot) == 0:
    print("None of the preferred indicator codes are available for plotting.")
else:
    plot_df = obs[obs["indicator_code"].isin(codes_to_plot)].dropna(subset=["observation_date", "value_numeric"]).copy()
    plot_df = plot_df.sort_values("observation_date")

    # Build a 'major events' subset to keep the overlay readable
    ev2 = get_events_timeline(df).copy()
    ev2["category"] = ev2["category"].astype(str)
    ev2["indicator"] = ev2["indicator"].astype(str)

    major_keywords = [
        "telebirr",
        "m-pesa",
        "mpesa",
        "interoper",
        "directive",
        "psp",
        "digital id",
        "fayda",
        "launch",
        "license",
        "regulation",
    ]

    ev2["is_major"] = (
        ev2["category"].str.contains("policy|regulation|directive|product|launch|infrastructure", case=False, na=False)
        | ev2["indicator"].str.contains("|".join(major_keywords), case=False, na=False)
    )
    major_ev = ev2[ev2["is_major"]].sort_values("observation_date")

    fig, ax = plt.subplots(figsize=(12, 6))

    for code in codes_to_plot:
        sub = plot_df[plot_df["indicator_code"] == code]
        # Use yearly mean to avoid double-plotting if multiple observations per year
        by_year = sub.groupby("year", as_index=False)["value_numeric"].mean().sort_values("year")
        ax.plot(by_year["year"], by_year["value_numeric"], marker="o", linewidth=2, label=code)

    # Event overlay (major only)
    y_max = ax.get_ylim()[1]
    for _, r in major_ev.iterrows():
        year = pd.to_datetime(r["observation_date"]).year
        ax.axvline(year, color="red", alpha=0.25, linestyle="--")
        ax.text(year, y_max, r["indicator"][:28] + "…", rotation=90, fontsize=8, va="top", ha="right")

    ax.set_xlabel("Year")
    ax.set_ylabel("Value (native units)")
    ax.set_title("Key indicators with major events overlaid")
    ax.grid(True, alpha=0.25)
    ax.legend(bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

    print("Major events included in overlay:")
    display(major_ev[["observation_date", "category", "indicator"]])

In [ ]:
# One row per year (mean across dates in that year) for cleaner correlation
obs_wide = obs.pivot_table(index="year", columns="indicator_code", values="value_numeric", aggfunc="mean")
numeric = obs_wide.select_dtypes(include=[np.number])
if numeric.shape[1] >= 2:
    corr = numeric.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="RdBu_r", center=0, square=True)
    plt.title("Correlation between indicators (across observation dates)")
    plt.tight_layout()
    plt.show()
    acc_corr = corr["ACC_OWNERSHIP"].drop("ACC_OWNERSHIP", errors="ignore").sort_values(ascending=False, key=lambda x: x.abs())
    print("Factors most correlated with ACC_OWNERSHIP:")
    print(acc_corr.head(10).to_string())

In [ ]:
if "USG_DIGITAL_PAY" in numeric.columns:
    use_corr = corr["USG_DIGITAL_PAY"].drop("USG_DIGITAL_PAY", errors="ignore").sort_values(ascending=False, key=lambda x: x.abs())
    print("Factors most correlated with USG_DIGITAL_PAY:")
    print(use_corr.head(10).to_string())
print("\nImpact_link records (event -> indicator):")
print(impact[["parent_id", "related_indicator", "impact_direction", "impact_magnitude", "impact_estimate", "lag_months"]].to_string(index=False))

---
## 7. Key Insights & Data Quality Assessment

## 7. Key Insights (linked to plots)

1. **Access improved substantially over the long run, but the most recent period shows slower momentum.**
   - See **Section 2.1** (level) and **Section 2.5** (annualized/YoY growth). The slope between the last two observation points is smaller than earlier jumps.

2. **The pace of change varies by survey interval, so “trend” depends heavily on which years you compare.**
   - See **Section 2.2** (change between survey years) and **Section 2.5** (annualized pp/year). Longer gaps can hide volatility and short-run accelerations/slowdowns.

3. **A persistent gender gap is visible in ownership (when gender-disaggregated values are available).**
   - See **Section 2.3**. The male series remains above the female series in the same years, consistent with structural access barriers.

4. **Mobile money penetration remains comparatively low relative to broader digital payment adoption.**
   - See **Section 3.1** (mobile money account %) vs **Section 3.2** (digital payment adoption %). This suggests many digital payments may be occurring through bank accounts/cards/other rails (not only mobile-money accounts), or that “account” vs “usage” definitions differ.

5. **Enablers improve faster than inclusion outcomes, implying non-infrastructure constraints matter.**
   - See **Section 4**. Mobile penetration / 4G coverage can rise while account ownership and usage rise more slowly, indicating barriers like onboarding/KYC, trust, affordability, and agent/cash-out constraints.

6. **Event timing can help interpret inflection points, but attribution is not causal.**
   - See **Section 5.2** and **5.3**. Some events align with changes in indicators, but events are not randomized and impacts may be lagged, overlapping, or confounded.

---
## Data limitations

- **Coverage / frequency**
  - Key survey indicators (e.g., Global Findex) appear only in a few years, so “2011–2024 trends” are based on sparse points.
  - Several infrastructure/enabler indicators are placeholders with limited temporal depth.

- **Confidence / comparability**
  - The dataset mixes **survey**, **operator/regulator**, and **estimated** values; confidence levels differ across sources.
  - Indicators may use different **denominators** and **definitions** (e.g., “has an account” vs “used in past 12 months”; operator “registered” vs survey “active/used”).

- **Bias and representativeness**
  - Survey-based measures can have sampling and response biases; operator metrics can overstate inclusion via multiple SIMs/accounts and do not always measure active use.
  - Observations are primarily **national-level** in this dataset, limiting insights on rural/urban, regional, or income-group disparities.

- **Events overlay limitations**
  - Event lists can be incomplete or subjective (selection/labeling). Overlays help contextualize timelines but should not be interpreted as proof of causality.